<a href="https://colab.research.google.com/github/dorsaalh/Cryptocurrency_Price_Prediction/blob/main/Candles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-binance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 2.2 MB/s 
     |████████████████████████████████| 288 kB 11.0 MB/s 
     |████████████████████████████████| 112 kB 59.8 MB/s 
     |████████████████████████████████| 749 kB 38.7 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2


In [ ]:
import binance
import pandas as pd
import datetime
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("/content/drive/My Drive/tweets.csv")

In [ ]:
data['account'].unique()

array(['Bloomberg', 'Documenting Bitcoin 📄', 'Pomp 🌪', 'Nick Szabo',
       'nic cubeter', 'Josh Olszewicz (I will not dm you)',
       'Alex Gladstein 🌋 ⚡', 'PlanB', 'Messari', 'Marty Bent', 'Andreas',
       'TOBTC', 'Michaël van de Poppe', 'Kraken Exchange', 'Symbol & NEM',
       'IOTA Engineering', 'Coinbase Institutional', 'Monero Outreach',
       'Coinbase Pro', 'Dogecoin', 'Elon Musk', 'CoinMarketCap',
       'Justin Sun 🅣🌞', 'Coinbase', 'Bitfinex', 'TRON Foundation',
       'Boba Network ($OMG)', 'Dash', 'Qtum', 'Stellar', 'LiteCoin News',
       'Litecoin', 'Ethereum Classic', 'Whale Alert', 'IOTA',
       'Waves Tech', 'Ripple', 'Bitcoin', 'Bitcoin News', 'Bittrex',
       'Neo Smart Economy', 'Ethereum', 'Monero || #xmr', 'vitalik.eth',
       'CZ 🔶 Binance', 'Binance'], dtype=object)

In [ ]:
sample_tweets = data[data['account'].isin(['Bitcoin News', 'Bloomberg', 'Nick Szabo', 'nic cubeter', 'TOBTC', 'Kraken Exchange',
                                           'CZ 🔶 Binance', 'Binance', 'Coinbase', 'Elon Musk', 'Coinbase Institutional', 'Ethereum Classic',
                                           'Coinbase Pro', 'LiteCoin News', 'Litecoin', 'Ethereum Classic', 'Documenting Bitcoin 📄', 'Stellar'])]

In [ ]:
crypto_name = { "Bitcoin News": "BTCUSDT",
               "Bloomberg": "BTCUSDT",
               "Nick Szabo": "BTCUSDT",
               "nic cubeter": "BTCUSDT",
               "TOBTC": "BTCUSDT",
               "Kraken Exchange": "BTCUSDT",
               "CZ 🔶 Binance": "BTCUSDT",
               "Binance": "BTCUSDT",
               "Coinbase": "BTCUSDT",
               "Elon Musk": "BTCUSDT",
               "Coinbase Institutional": "BTCUSDT",
               "Coinbase Pro": "BTCUSDT",
               "Litecoin": "LTCUSDT",
               "LiteCoin News": "LTCUSDT",
               "Ethereum Classic": 'ETHUSDT',
               'Documenting Bitcoin 📄': 'BTCUSDT',
               "Stellar":'XLMUSDT'}

In [ ]:
year = []
month = []
day = []
hour = []
minute = []

for t in sample_tweets.time:
  date_format = time.strptime(t, "%a %b %d %H:%M:%S +0000 %Y")
  year.append(date_format.tm_year)
  month.append(date_format.tm_mon)
  day.append(date_format.tm_mday)
  hour.append(date_format.tm_hour)
  minute.append(date_format.tm_min)

In [ ]:
sample_tweets['year'] = year
sample_tweets['month'] = month
sample_tweets['day'] = day
sample_tweets['hour'] = hour
sample_tweets['minute'] = minute

sample_tweets.drop('time', axis = 1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
sample_tweets

,Unnamed: 0,account,text,year,month,day,hour,minute
0,0,Bloomberg,"“Needless to say, this is an aggressive deal” ...",2021,11,5,12,40
1,1,Bloomberg,RT @BW: NEW: Energy traders’ big bets could be...,2021,11,5,12,40
2,2,Bloomberg,"RT @ArmstrongDrew: Well, Pfizer's having a hel...",2021,11,5,12,39
3,3,Bloomberg,RT @markets: U.S. stock futures rise https://t...,2021,11,5,12,39
4,4,Bloomberg,The U.S. labor market got back on track last m...,2021,11,5,12,39
...,...,...,...,...,...,...,...,...
8870,8870,Binance,009🔺 @BITCOIN_AL_DIA https://t.co/UtwYD9CnlD,2021,10,26,13,2
8871,8871,Binance,008🔺 @REALDIEGOVALDEZ https://t.co/h25WTLtegd,2021,10,26,13,2
8872,8872,Binance,007🔺 @kursadbucak https://t.co/PVIlyhrbMj,2021,10,26,13,2
8873,8873,Binance,006🔺 @healthy_pockets https://t.co/fitDIMC8ke,2021,10,26,13,2


In [ ]:
client = binance.Client()

In [ ]:
def profit_calculate_1hour(index):
  start = datetime.datetime(sample_tweets.iloc[index].year, sample_tweets.iloc[index].month,
                            sample_tweets.iloc[index].day, sample_tweets.iloc[index].hour, sample_tweets.iloc[index].minute)
  end = start + datetime.timedelta(hours=6)
  klines = client.get_historical_klines(crypto_name[sample_tweets.iloc[index].account], client.KLINE_INTERVAL_5MINUTE,
                                        start_str=str(start), end_str=str(end))
  #convert candles info, from list to DataFrame
  klines_df_1 = pd.DataFrame(klines, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
                                          'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
  flag = 0
  
  if (len(klines_df_1) == 72):
    flag = 1

  if (flag == 0):
    return 1000, 1000, 1000, 1000, 1000

  
  candle_number = len(klines_df_1) - 1

  #convert time frame from 5Min to 1Hour
  open_time = []
  close_time = []
  open = []
  close = []
  high = []
  low = []

  for i in range(0, candle_number, 12):
    open_time.append( klines_df_1['Open time'][i] )
    close_time.append( klines_df_1['Close time'][np.min([i+11, candle_number])] )
    open.append( klines_df_1['Open'][i] )
    close.append( klines_df_1['Close'][np.min([i+11, candle_number])] )
    high.append( klines_df_1['High'][i:np.min([i+11, candle_number])].max() )
    low.append( klines_df_1['Low'][i:np.min([i+11, candle_number])].min() )

  #create new DataFrame of candles info in 1Hour time frame
  gathering_data = {'Open_time':open_time ,
                    'Close_time':close_time ,
                    'Open':open , 
                    'Close':close ,
                    'High':high ,
                    'Low':low}
  
  candles_data = pd.DataFrame(gathering_data)

  #price in tweet moment
  end_2 = start + datetime.timedelta(seconds= 1)
  tweet_moment = client.get_historical_klines(crypto_name[sample_tweets.iloc[index].account], client.KLINE_INTERVAL_1MINUTE,
                                              start_str=str(start), end_str=str(end_2))
  print(index)
  #print(tweet_moment)
  tweet_moment_price = float(tweet_moment[0][1])

  max_1 = candles_data['High'].max()
  min_1 = candles_data['Low'].min()
  highest_1 = float(max_1)
  lowest_1 = float(min_1)
  profit_percent_1 = (highest_1 - tweet_moment_price) / tweet_moment_price
  loss_percent_1 = (lowest_1 - tweet_moment_price) / tweet_moment_price

  max_index_1 = candles_data[candles_data['High']== max_1].index[0]
  min_index_1 = candles_data[candles_data['Low']== min_1].index[0]

  return (profit_percent_1*100), (loss_percent_1*100), max_index_1 + 1, min_index_1 + 1, flag

In [ ]:
profit_1 = []
loss_1 =[]
Highest_index_1= []
Lowest_index_1= []
counter = 0

for i in range(len(sample_tweets)):
  p_1, l_1, highest_i_1, lowest_i_1, flag = profit_calculate_1hour(i)
  profit_1.append(p_1)
  loss_1.append(l_1)
  Highest_index_1.append(highest_i_1)
  Lowest_index_1.append(lowest_i_1)
  if(flag == 1):
    counter = counter +1

sample_tweets['profit_1Hour'] = profit_1
sample_tweets['loss_1Hour'] = loss_1
sample_tweets['highest_index_1Hour'] = Highest_index_1
sample_tweets['lowest_index_1Hour'] = Lowest_index_1

In [ ]:
sample_tweets.shape

(3386, 12)

In [ ]:
sample_tweets = sample_tweets[sample_tweets['lowest_index_1Hour']!=1000]

In [ ]:
sample_tweets.reset_index(drop=True, inplace=True)
sample_tweets

,Unnamed: 0,account,text,year,month,day,hour,minute,profit_1Hour,loss_1Hour,highest_index_1Hour,lowest_index_1Hour
0,2,Bloomberg,"RT @ArmstrongDrew: Well, Pfizer's having a hel...",2021,11,5,12,39,0.729112,-1.169577,2,4
1,3,Bloomberg,RT @markets: U.S. stock futures rise https://t...,2021,11,5,12,39,0.729112,-1.169577,2,4
2,4,Bloomberg,The U.S. labor market got back on track last m...,2021,11,5,12,39,0.729112,-1.169577,2,4
3,5,Bloomberg,Follow the latest on the October jobs report h...,2021,11,5,12,38,0.713624,-1.184774,2,4
4,6,Bloomberg,It will take years to fully understand the pan...,2021,11,5,12,38,0.713624,-1.184774,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2547,8870,Binance,009🔺 @BITCOIN_AL_DIA https://t.co/UtwYD9CnlD,2021,10,26,13,2,0.384693,-1.170566,3,2
2548,8871,Binance,008🔺 @REALDIEGOVALDEZ https://t.co/h25WTLtegd,2021,10,26,13,2,0.384693,-1.170566,3,2
2549,8872,Binance,007🔺 @kursadbucak https://t.co/PVIlyhrbMj,2021,10,26,13,2,0.384693,-1.170566,3,2
2550,8873,Binance,006🔺 @healthy_pockets https://t.co/fitDIMC8ke,2021,10,26,13,2,0.384693,-1.170566,3,2


In [ ]:
sample_tweets.shape

(2552, 12)

In [ ]:
label = [0]*len(sample_tweets)    #0:neutral  1:positive  2:negative

for i in range(len(sample_tweets)):
  if ([sample_tweets[sample_tweets.index==i]['profit_1Hour'] >= 1][0].values[0]):
    if ([sample_tweets[sample_tweets.index==i]['loss_1Hour'] > -1][0].values[0]):
      label[i] = 1
    else:
      if ([sample_tweets[sample_tweets.index==i]['highest_index_1Hour']][0].values[0] > [sample_tweets[sample_tweets.index==i]['lowest_index_1Hour']][0].values[0]):
        label[i] = 1
      else:
        label[i] = 2

  else:
    if ([sample_tweets[sample_tweets.index==i]['loss_1Hour'] <= -1][0].values[0]):
      label[i] = 2

In [ ]:
d = { 'text': sample_tweets.text, 'account':sample_tweets.account, 'label': label}
simple_data = pd.DataFrame(data=d)

In [ ]:
neutral_num = len(simple_data[simple_data.label==0])
negative_num = len(simple_data[simple_data.label==2])
positive_num = len(simple_data[simple_data.label==1])
print(f'number of neutral:{neutral_num}')
print(f'number of negative:{negative_num}')
print(f'number of positive:{positive_num}')

number of neutral:1522
number of negative:486
number of positive:544


In [ ]:
simple_data

,text,account,label
0,"RT @ArmstrongDrew: Well, Pfizer's having a hel...",Bloomberg,0
1,RT @markets: U.S. stock futures rise https://t...,Bloomberg,0
2,The U.S. labor market got back on track last m...,Bloomberg,0
3,Follow the latest on the October jobs report h...,Bloomberg,0
4,It will take years to fully understand the pan...,Bloomberg,0
...,...,...,...
2547,009🔺 @BITCOIN_AL_DIA https://t.co/UtwYD9CnlD,Binance,0
2548,008🔺 @REALDIEGOVALDEZ https://t.co/h25WTLtegd,Binance,0
2549,007🔺 @kursadbucak https://t.co/PVIlyhrbMj,Binance,0
2550,006🔺 @healthy_pockets https://t.co/fitDIMC8ke,Binance,0


In [ ]:
# simple_data.to_csv('data_for_finbert.csv')
# !cp data_for_finbert.csv "drive/My Drive/"